In [1]:
import os
import pandas as pd
import duckdb
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
load_dotenv()

# Debug: Print ENV vars
print("🔐 Azure ENV Check:")
print("API Key Present:", "AZURE_OPENAI_API_KEY" in os.environ)
print("Endpoint:", os.getenv("AZURE_OPENAI_ENDPOINT"))
print("Deployment Name:", os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"))

# Step 1: Setup Azure OpenAI Chat LLM
llm = AzureChatOpenAI(
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_version="2023-05-15",
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    temperature=0,
)

🔐 Azure ENV Check:
API Key Present: True
Endpoint: https://replica.openai.azure.com/
Deployment Name: o4-mini


In [4]:
print("✅ AzureChatOpenAI initialized")

✅ AzureChatOpenAI initialized


In [5]:
file_path = "netflix_titles.csv"  # replace with your CSV path
try:
    df = pd.read_csv(file_path, encoding="utf-8", encoding_errors="ignore")
    print("✅ CSV loaded successfully:", df.shape)
except Exception as e:
    print("❌ Failed to read CSV:", e)
    raise

# Handle missing or invalid values
df.dropna(how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.fillna(value={col: 'unknown' if df[col].dtype == 'O' else 0 for col in df.columns}, inplace=True)

print("🧹 Data cleaned. Shape:", df.shape)

✅ CSV loaded successfully: (8807, 12)
🧹 Data cleaned. Shape: (8807, 12)


In [6]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,unknown,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",unknown,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,unknown,unknown,unknown,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [7]:
import tempfile
# with tempfile.NamedTemporaryFile(suffix=".duckdb") as tmp_db:
con = duckdb.connect(database=':memory:')
con.register("dataset", df)
print("🦆 DuckDB registered with table 'dataset'")

# Step 4: Define prompt template
prompt = PromptTemplate.from_template(
    "You are an expert data analyst. Write a SQL query for DuckDB based on this question:\n\n"
    "Question: {question}\n"
    "Only return the SQL query without explanation.\n"
    "DuckDB Table name: dataset\n"
    "Columns: {columns}"
)

chain = LLMChain(llm=llm, prompt=prompt)
print("🔗 LLMChain ready")

# Step 5: Ask a question
user_question = "How many movies were released in 2020?"  # example
print("🧠 User Question:", user_question)

generated_sql = chain.run({
    "question": user_question,
    "columns": ", ".join(df.columns)
})

print("📝 Generated SQL:\n", generated_sql)

cleaned_sql=generated_sql.strip().replace("```sql","").replace("```","").replace('"""','').replace("'''","")

# Step 6: Execute query
try:
    result_df = con.execute(cleaned_sql).fetchdf()
    print("✅ Query executed successfully:\n", result_df)
except Exception as e:
    print("❌ SQL Execution Error:", e)

🦆 DuckDB registered with table 'dataset'
🔗 LLMChain ready
🧠 User Question: How many movies were released in 2020?


InvalidRequestError: Model o4-mini is enabled only for api versions 2024-12-01-preview and later